In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
! pip install transformers datasets accelerate peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

In [3]:
data = load_dataset("its-myrto/fitness-question-answers")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/203 [00:00<?, ?B/s]

conversational_dataset.csv:   0%|          | 0.00/289k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/965 [00:00<?, ? examples/s]

In [4]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Question', 'Answer'],
        num_rows: 965
    })
})

In [5]:
df = data["train"].to_pandas()
df

,Unnamed: 0,Question,Answer
0,0,can you recommend effective core ab exercises?,"sure! planks, bicycle crunches, and leg raises..."
1,1,how can i incorporate physical activity into m...,take the stairs instead of the elevator whenev...
2,2,why is physical activity important?,regular physical activity can help maintain yo...
3,3,how can i manage stress and maintain a healthy...,move your body. exercise regularly and engage ...
4,4,i have a busy travel schedule. how can i maint...,"travel can disrupt routines, but it's manageab..."
...,...,...,...
960,960,is 10 minutes of stretching enough?,"if youre warming up for a workout, 5 to 10 min..."
961,961,should stretching be painful?,its normal to feel mild to moderate discomfort...
962,962,is there a wrong way to stretch?,"regularly stretching to the point of pain, try..."
963,963,is it okay to stretch every day?,"not only is it okay to stretch every day, its ..."


In [6]:
df["Question"][0]

'can you recommend effective core ab exercises?'

In [7]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, shuffle=True, random_state=0)
train_data

,Unnamed: 0,Question,Answer
648,648,what's the biggest mistake people make at the ...,most people who start a fitness routine don't ...
196,196,how many times should i do each stretch?,each stretch should be repeated for the desire...
342,342,can i exercise with limited space?,"absolutely! bodyweight exercises like squats, ..."
312,312,is it safe to exercise if i have health condit...,consult your doctor before starting any exerci...
62,62,how does physical inactivity impact health?,physical inactivity is a leading risk factor f...
...,...,...,...
835,835,what are kegel exercises?,"kegel exercise, also known as pelvic floor exe..."
192,192,what's most important for achieving a healthy ...,"the key is to reduce sedentary time, stay acti..."
629,629,i exercise five days a week. 10 minutes stretc...,only doing arm and back weights means that you...
559,559,what is a set?,a set is a certain number of repetitions. for ...


In [8]:
train_data.to_parquet("/content/drive/MyDrive/GymBoy2/train.parquet", index=False)
test_data.to_parquet("/content/drive/MyDrive/GymBoy2/test.parquet", index=False)

# **Chatbot**

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
import pandas as pd


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [10]:
train_df = pd.read_parquet("/content/drive/MyDrive/GymBoy2/train.parquet")
test_df = pd.read_parquet("/content/drive/MyDrive/GymBoy2/test.parquet")
train_data = Dataset.from_pandas(train_df)
test_data = Dataset.from_pandas(test_df)

In [11]:
train_data

Dataset({
    features: ['Unnamed: 0', 'Question', 'Answer'],
    num_rows: 772
})

In [12]:
model_id="google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [13]:
def preprocess_function(sample,padding="max_length"):
    model_inputs = tokenizer(sample["Question"], max_length=256, padding=padding, truncation=True)
    labels = tokenizer(sample["Answer"], max_length=256, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [14]:
train_tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
test_tokenized_dataset = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)
print(f"Keys of tokenized dataset: {list(train_tokenized_dataset.features)}")

Map:   0%|          | 0/772 [00:00<?, ? examples/s]

Map:   0%|          | 0/193 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [15]:
lora_config = LoraConfig(
 r=16,
 lora_alpha=32,
 target_modules=["q", "v"],
 lora_dropout=0.1,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)


In [16]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 787,868,672 || trainable%: 0.5989


In [17]:
label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
output_dir="/content/drive/MyDrive/GymBoy2/lora-flan-t5-large-chat2"
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    learning_rate=1e-3,
    num_train_epochs=1,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="epoch",
    save_strategy="epoch",
    report_to="tensorboard",
    push_to_hub = True
)


In [20]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_tokenized_dataset,
)

In [21]:
model.config.use_cache = False

In [22]:
trainer.train()
peft_save_model_id="lora-flan-t5-large-chat"
trainer.model.save_pretrained(peft_save_model_id, push_to_hub=True)
tokenizer.save_pretrained(peft_save_model_id, push_to_hub=True)
trainer.model.base_model.save_pretrained(peft_save_model_id, push_to_hub=True)

Step,Training Loss
193,2.450900


No files have been modified since last commit. Skipping to prevent empty commit.


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.15G [00:00<?, ?B/s]

In [ ]:
! cp -r /content/drive/MyDrive/GymBoy/lora-flan-t5-large-chat/ /content/drive/MyDrive/GymBoy/